**NB : Ce code doit etre importé et exécuté sur Kaggle.com, il génére des erreurs sur les autres plateformes (comme Jupyter, Spider ...)**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/model-joblib/model_joblib
/kaggle/input/movielens-20m-dataset/tag.csv
/kaggle/input/movielens-20m-dataset/movie.csv
/kaggle/input/movielens-20m-dataset/link.csv
/kaggle/input/movielens-20m-dataset/genome_scores.csv
/kaggle/input/movielens-20m-dataset/rating.csv
/kaggle/input/movielens-20m-dataset/genome_tags.csv


In [2]:
!pip install pyspark

     |████████████████████████████████| 215.7MB 56kB/s s eta 0:00:01
     |████████████████████████████████| 204kB 41.5MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216131250 sha256=2e3f66d7061b384a67889eee324f58666edd6de1fd87c90c35222762dfbff4ef
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.sql("select 'spark' as hello")

df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [5]:
import pandas as pd

In [6]:
#genome_scores = pd.read_csv("../input/movielens-20m-dataset/genome_scores.csv")
#genome_tags = pd.read_csv("../input/movielens-20m-dataset/genome_tags.csv")
#link = pd.read_csv("../input/movielens-20m-dataset/link.csv")
#movie = pd.read_csv("../input/movielens-20m-dataset/movie.csv")
#rating = pd.read_csv("../input/movielens-20m-dataset/rating.csv")
#tag = pd.read_csv("../input/movielens-20m-dataset/tag.csv")

In [7]:
# La table des notes des films
movie_ratings = spark.read.csv("../input/movielens-20m-dataset/rating.csv", inferSchema=True, header=True)
movie_ratings = movie_ratings.select('userId', 'movieId', 'rating')

In [8]:
type(movie_ratings)

pyspark.sql.dataframe.DataFrame

In [9]:
# Cette contient environs 20 milions de lignes, on prend juste les 1 milions premiers
movie_ratings = movie_ratings.head(1000000)

In [10]:
type(movie_ratings)

list

In [11]:
# l'étape precedante nous retourne une liste, on crée une liste des noms des colonnes avec lesquels on vas structurer la liste movie_ratings
columns = list(['userId', 'movieId', 'rating'])

In [12]:
# Création d'une dataframe à partir de la liste movie_ratings avec les noms de colonnes columns
movie_ratings = spark.createDataFrame(movie_ratings, columns)

In [13]:
movie_ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      2|   3.5|
|     1|     29|   3.5|
|     1|     32|   3.5|
|     1|     47|   3.5|
|     1|     50|   3.5|
|     1|    112|   3.5|
|     1|    151|   4.0|
|     1|    223|   4.0|
|     1|    253|   4.0|
|     1|    260|   4.0|
|     1|    293|   4.0|
|     1|    296|   4.0|
|     1|    318|   4.0|
|     1|    337|   3.5|
|     1|    367|   3.5|
|     1|    541|   4.0|
|     1|    589|   3.5|
|     1|    593|   3.5|
|     1|    653|   3.0|
|     1|    919|   3.5|
+------+-------+------+
only showing top 20 rows



In [14]:
movie_ratings.count()

1000000

In [15]:
# Alternating Least Squares (Alternance des moindres carrés)
from pyspark.ml.recommendation import ALS
# RegressionEvaluator pour évaluer la performance du modèle ALS
from pyspark.ml.evaluation import RegressionEvaluator
# CrossValidator pour diviser la dataset en training and testing
# ParamGridBuilder pour affiner les paramètres de notre modèle
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [16]:
# Création de training set et test set
(training, test) = movie_ratings.randomSplit([0.8, 0.2])

In [17]:
# Création du modèle ALS (Alternating Least Saqures)
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating', coldStartStrategy='drop', nonnegative=True)

# nonnegative=True : car on veut pas qu'il nous retourne des valeurs négatives

In [18]:
# Régler le modèle en utilisant ParamGridBuilder
param_grid = ParamGridBuilder().addGrid(als.rank, [12, 13, 14]).addGrid(als.maxIter, [18, 19, 20]).addGrid(als.regParam, [.17, .18, .19]).build()

# On le donne :
# les paramètres des matrices U et P
# max iterations qui disent à Spark combien de fois alterner entre U et P pour minimiser l'erreur
# le paramètre de régularisation pour empêcher ALS de sur-adapter aux données (overfitting)

In [19]:
# Définir l'Évaluateur de régression, qui attend la prédiction des colonnes d'entrée
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# predictionCol='prediction' : le nom de la colonne des prédictions

In [20]:
# Construction de cross validation
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# estimator=als : pour utiliser le modèle de ALS

In [21]:
# Entrainer le modèle avec les données d'entraînement
model = cv.fit(training)

In [22]:
# Extraire le meilleur modèle de l'exercice de tournage à l'aide de ParamGridBuilder
best_model = model.bestModel

In [23]:
# Générer des prédictions et évaluer à l'aide de RMSE
predictions = best_model.transform(test)
rmse = evaluator.evaluate(predictions)

# rmse : Écart quadratique moyen (Root-mean-square deviation)

In [24]:
# Afficher les métriques d'évaluation du modèle
print("RMSE = " + str(rmse))

RMSE = 0.8549127621467842


In [25]:
# Comparer les prédictions des évaluations des utilisateurs (ratings) avec les évaluations réels
predictions.sort('userId', 'rating').show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|   5040|   3.0|  2.937481|
|     1|   1525|   3.0| 2.6540728|
|     1|   4915|   3.0| 2.7165065|
|     1|   5679|   3.5|  3.408867|
|     1|   8690|   3.5| 3.3563516|
|     1|   4105|   3.5| 3.5295167|
|     1|   7247|   3.5| 3.4042552|
|     1|   2683|   3.5|  3.200656|
|     1|   2692|   3.5|  3.821571|
|     1|   7387|   3.5| 3.5542927|
|     1|   5146|   3.5| 3.4806368|
|     1|    919|   3.5| 3.7107344|
|     1|   2664|   3.5|  3.437896|
|     1|     47|   3.5|  3.902289|
|     1|   4754|   4.0|  3.596634|
|     1|   8961|   4.0| 3.8429916|
|     1|   7389|   4.0| 2.8628192|
|     1|   1219|   4.0| 3.8183458|
|     1|   2288|   4.0|  3.717929|
|     1|    318|   4.0| 4.2863574|
+------+-------+------+----------+
only showing top 20 rows



In [26]:
# Génerer les prédictions des évaluations de tous les utilisateurs (Recommander 10 films pour chaque utilisateur)
users_recommendations = best_model.recommendForAllUsers(10)

In [27]:
users_recommendations.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|  1580|[[59295, 4.565479...|
|  4900|[[51455, 5.647877...|
|  5300|[[82931, 6.035058...|
|  6620|[[88678, 5.574956...|
|   471|[[95776, 4.997751...|
|  1591|[[26334, 5.478696...|
|  4101|[[82931, 5.202222...|
|  1342|[[82931, 5.777525...|
|  2122|[[39768, 4.855009...|
|  2142|[[88678, 5.679757...|
|   463|[[727, 5.639211],...|
|   833|[[95776, 5.401778...|
|  5803|[[95776, 6.342136...|
|  3794|[[95776, 5.604387...|
|  6654|[[95776, 6.47748]...|
|  1645|[[727, 5.3439198]...|
|  3175|[[95776, 6.269581...|
|  4935|[[82931, 4.689534...|
|   496|[[82931, 5.942974...|
|  2366|[[95776, 5.859939...|
+------+--------------------+
only showing top 20 rows



In [28]:
# SQLContext : Le point d'entrée pour travailler avec des données structurées (lignes et colonnes) dans Spark
from pyspark.sql import SQLContext

sqlContext = SQLContext(spark)

In [29]:
# Ajout de la table des films pour joindre les noms des films avec la table de recommendations en résultat

In [30]:
movieDF = spark.read.csv("../input/movielens-20m-dataset/movie.csv", inferSchema=True, header=True)

In [31]:
movieDF.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [38]:
# Pour faciliter l'affichage de users_recommendations

def get_recs_for_user(recs):
    recs = recs.select("userId", "recommendations.movieId", "recommendations.rating")
    movies = recs.select("movieId").toPandas().iloc[:, 0].values
    ratings = recs.select("rating").toPandas().iloc[:, 0].values
    userIds = recs.select("userId").toPandas()
    ratings_matrix = pd.DataFrame(movies, columns=['movieId'])
    #ratings_matrix['userId'] = userIds
    ratings_matrix.insert(0, 'userId', userIds)
    ratings_matrix['ratings'] = ratings
    ratings_matrix_ps = sqlContext.createDataFrame(ratings_matrix)
    return ratings_matrix_ps

In [39]:
users_recs = get_recs_for_user(users_recommendations)

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:41571)

In [ ]:
type(users_recs)

In [ ]:
users_recs.show()

In [ ]:
# Extraire de users_recommendations les recommandations pour un utilisateur spécifique
user_id = input("Donner l'id de l'utilisateur : ")
user_recs = users_recs.filter("userId="+user_id)

In [ ]:
user_recs.show()

In [ ]:
# Pour changer l'affichage de la liste des ids des films

z = []

for k,row in user_recs.toPandas().iterrows():
    for j in list(np.array(row.movieId).flat):
        z.append({'userId':row.userId, 'movieId':j})

user_recs = spark.createDataFrame(pd.DataFrame(z))

In [ ]:
# Joindre la dataframe des films recommandés pour l'utilisateur avec leurs titres et genres

user_recs = user_recs.join(movieDF, on='movieId')

In [ ]:
# Pour échanger les indexs de 'userId' et 'movieId'

user_recs = user_recs['userId', 'movieId', 'title', 'genres']

In [ ]:
# Affichage de la table de recommendation pour l'utilisateur demandé
user_recs.show()

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 35604)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/opt/conda/lib/python3.6/site-packages/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/opt/conda/lib/python3.6/site-packages/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/opt/conda/lib/python3.6/site-packages/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(sel